In [1]:
import os
from dotenv import load_dotenv
import autogen
import chromadb
from pathlib import Path
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
from autogen.retrieve_utils  import query_vector_db

load_dotenv()

True

In [2]:
# CREATING VECTOR STORE
docs_path = Path(os.getcwd(), 'docs')
client = chromadb.PersistentClient(path=f"{os.getcwd()}/chromadb")

In [3]:
# CONFIGURATIONS

config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='../../.env',
    model_api_key_map={
        "gpt-4-1106-preview": "OPENAI_API_KEY",
    },
    filter_dict={
        "model": {
            "gpt-4-1106-preview",
        }
    }
)

query_vector_db_tool_config = {
    "name": "query_vector_db",
    "description": "Function to query the Chroma vector database.",
    "parameters": {
        "type": "object",
        "properties": {
            "query_texts": {"type": "array", "items": {"type": "string"}},
            "n_results": {"type": "integer"}
        },
        "required": ["query_texts"]
    }
}

llm_config = {
    "config_list": config_list,
    "assistant_id": None,
    "tools": [
        {
            "type": "function",
            "function": query_vector_db_tool_config
        }
    ],
    "model": "gpt-4-1106-preview"
}

In [4]:
RAG_ASSISTANT_SYSTEM_MESSAGE = """You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:

Step 1: you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2: you reply based on the intent.

Use the provided tool `query_vector_db` to get the context from a Chroma vectorstore.
If user's intent is code generation you must follow the formats below to write your code:
```language
# your code
```

If user's intent is question answering, you must give as short an answer as possible.
Use the provided tool `query_vector_db` to get the context.
"""

In [ ]:
# INSTANTIATE AGENTS

sql_assistant = GPTAssistantAgent(
    name="Chroma_SQL_Assistant",
    instructions=RAG_ASSISTANT_SYSTEM_MESSAGE,
    llm_config=llm_config
)

sql_assistant.register_function(
    function_map={
        "query_vector_db": lambda query_texts, n_results=10: query_vector_db(client, query_texts, n_results)
    }
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    code_execution_config={
        "work_dir": "./autogen_results",
        "use_docker": False
    },
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3
)

initial_message = "How can you use Chainlit with Autogen"
user_proxy.initiate_chat(sql_assistant, message=initial_message)
